# TP - Prétraitement des données - Achats alimentaires

**Objectif**: Nettoyer et préparer le jeu de données bruité `donnees_brutes_achats.xlsx` pour le rendre exploitable.

**Étapes**:
1. Importer les données
2. Diagnostiquer la qualité des données
3. Gérer les doublons
4. Uniformiser les chaînes de caractères
5. Harmoniser les catégories
6. Unifier le format des dates
7. Gérer les valeurs manquantes
8. Détecter et traiter les valeurs aberrantes
9. Supprimer la colonne 'Notes'
10. Exporter la table finale propre


# **Rapport de nettoyage**

### Problèmes détectés:
1. **Doublons**: Présence de lignes dupliquées (exactes et par TransactionID)
2. **Chaînes non uniformisées**: Espaces parasites, casse incohérente, synonymes
3. **Catégories incohérentes**: Variations orthographiques et formats différents
4. **Dates hétérogènes**: Formats de dates variés
5. **Valeurs manquantes**: Dans les colonnes Quantité et Prix
6. **Valeurs aberrantes**: Quantités négatives/excessives, prix à 999
7. **Produits invalides**: Lignes avec '—' ou vides

### Choix de nettoyage:
- **Doublons**: Suppression des doublons exacts puis par TransactionID (conservation de la première occurrence)
- **Uniformisation**: Trim des espaces, conversion en minuscules, normalisation des accents
- **Synonymes**: Mapping manuel des variantes (pates→pâtes, tomato→tomate, etc.)
- **Catégories**: Harmonisation via mapping (epicerie→épicerie, fruits-legumes→fruits & légumes)
- **Dates**: Conversion avec `pd.to_datetime(dayfirst=True)` au format YYYY-MM-DD
- **Valeurs manquantes**: Imputation par la **médiane** (plus robuste que la moyenne face aux outliers)
- **Aberrants**: 
  - Quantités < 0 ou > 100: remplacement par la médiane
  - Prix > 100: remplacement par la médiane
  - Produits invalides: suppression des lignes
- **Colonne Notes**: Supprimée (non utilisée pour l'analyse)

### Impact:
Les statistiques détaillées (nombre de lignes supprimées, valeurs modifiées) sont affichées dans les cellules ci-dessus.

# **Nettoyage des données**
## 1. Import des bibliothèques et des données

In [1]:
import pandas as pd
import numpy as np

# Import data
df = pd.read_excel('donnees_brutes_achats.xlsx')

# Preview of dataset
print(f"Nombre total de lignes: {len(df)}")
print("\nPremières lignes du dataset:")
df.head(10)

Nombre total de lignes: 51

Premières lignes du dataset:


,TransactionID,Produit,Quantité,Prix,Catégorie,Date,Notes
0,1,Pain,1.0,1.20,Boulangerie,2025-09-01,NaN
1,2,Lait,2.0,0.95,Laitage,01/09/2025,NaN
2,3,Beurre,1.0,2.80,Laitage,2025/09/01,NaN
3,4,Tomate,3.0,1.99,Fruits & Légumes,2025-09-02,NaN
4,5,tomato,2.0,2.10,fruits et legumes,02-09-2025,NaN
5,6,Pâtes,1.0,0.89,Épicerie,2025-09-02,promo? 2 pour 1
6,7,pates,2.0,0.89,epicerie,02/09/25,NaN
7,8,Riz,1.0,1.10,Épicerie,2025-09-03,NaN
8,9,Riz,5.0,1.10,épicerie,2025-09-03,NaN
9,10,Yaourt,6.0,0.45,Laitage,2025-09-03,NaN


## 2. Diagnostic de la qualité des données

In [2]:
# General information
print("=== Informations sur le dataset ===")
df.info()

=== Informations sur le dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  51 non-null     int64  
 1   Produit        51 non-null     object 
 2   Quantité       49 non-null     float64
 3   Prix           49 non-null     float64
 4   Catégorie      49 non-null     object 
 5   Date           51 non-null     object 
 6   Notes          4 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 2.9+ KB


In [3]:
# Statistical description
print("\n=== Description statistique ===")
df.describe(include='all')


=== Description statistique ===


,TransactionID,Produit,Quantité,Prix,Catégorie,Date,Notes
count,51.000000,51,49.000000,49.000000,49,51,4
unique,NaN,40,NaN,NaN,22,29,4
top,NaN,Beurre,NaN,NaN,Laitage,2025-09-06,promo? 2 pour 1
freq,NaN,4,NaN,NaN,9,4,1
mean,25.803922,NaN,23.061224,22.349184,NaN,NaN,NaN
std,14.593176,NaN,142.494943,142.437880,NaN,NaN,NaN
min,1.000000,NaN,-1.000000,0.000000,NaN,NaN,NaN
25%,13.500000,NaN,1.000000,0.900000,NaN,NaN,NaN
50%,26.000000,NaN,2.000000,1.750000,NaN,NaN,NaN
75%,38.500000,NaN,4.000000,2.800000,NaN,NaN,NaN


In [4]:
# Missing values
print("\n=== Valeurs manquantes ===")
missing_values = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Nombre de valeurs manquantes': missing_values,
    'Pourcentage': missing_percent
})
print(missing_df[missing_df['Nombre de valeurs manquantes'] > 0])


=== Valeurs manquantes ===
           Nombre de valeurs manquantes  Pourcentage
Quantité                              2     3.921569
Prix                                  2     3.921569
Catégorie                             2     3.921569
Notes                                47    92.156863


In [5]:
# Duplicates
print("\n=== Analyse des doublons ===")
nb_duplicates = df.duplicated().sum()
print(f"Nombre de doublons exacts: {nb_duplicates}")

# TransactionID duplicates
nb_id_duplicates = df.duplicated(subset=['TransactionID']).sum()
print(f"Nombre de doublons par TransactionID: {nb_id_duplicates}")


=== Analyse des doublons ===
Nombre de doublons exacts: 1
Nombre de doublons par TransactionID: 1


In [6]:
# Unique values
print("\n=== Valeurs uniques ===")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valeurs uniques")
    if col in ['Produit', 'Catégorie'] and df[col].nunique() < 50:
        print(f"  Exemples: {df[col].unique()[:10]}")


=== Valeurs uniques ===
TransactionID: 50 valeurs uniques
Produit: 40 valeurs uniques
  Exemples: ['Pain' 'Lait' 'Beurre' 'Tomate' 'tomato' 'Pâtes' 'pates' 'Riz' 'Riz '
 'Yaourt']
Quantité: 12 valeurs uniques
Prix: 34 valeurs uniques
Catégorie: 22 valeurs uniques
  Exemples: ['Boulangerie' 'Laitage' 'Fruits & Légumes' 'fruits et legumes' 'Épicerie'
 'epicerie' 'épicerie' 'laitage' 'Œufs & Ovoproduits' 'oeufs']
Date: 29 valeurs uniques
Notes: 4 valeurs uniques


## 3. Gestion des doublons

In [7]:
# Save original df lenght
len_original = len(df)
print(f"Nombre de lignes avant suppression des doublons: {len_original}\n")

# Delete duplicates
df.drop_duplicates(inplace=True)
print(f"Nombre de lignes après suppression des doublons exacts: {len(df)}")
print(f"Doublons exacts supprimés: {len_original - len(df)}\n")

# Delete ID duplicates (keep first)
len_before_id = len(df)
df.drop_duplicates(subset=['TransactionID'], inplace=True)
print(f"Nombre de lignes après suppression des doublons par TransactionID: {len(df)}")
print(f"Doublons par TransactionID supprimés: {len_before_id - len(df)}")

Nombre de lignes avant suppression des doublons: 51

Nombre de lignes après suppression des doublons exacts: 50
Doublons exacts supprimés: 1

Nombre de lignes après suppression des doublons par TransactionID: 50
Doublons par TransactionID supprimés: 0


## 4. Uniformisation des chaînes de caractères

In [8]:
# Trim spaces, and convert into lower case
print("=== Uniformisation des chaînes ===")
print(f"\nExemples de produits avant nettoyage:")
print(df['Produit'].head(20).tolist())

=== Uniformisation des chaînes ===

Exemples de produits avant nettoyage:
['Pain', 'Lait', 'Beurre', 'Tomate', 'tomato', 'Pâtes', 'pates', 'Riz', 'Riz ', 'Yaourt', 'yaourts', 'Oeufs', 'oeuf', 'Poulet', 'Poisson', 'Banane', 'bananes', 'Pomme', 'Pommes', 'Concombre']


In [9]:
def find_whitespace_in_values(df):
    """Find columns with leading/trailing whitespace in values"""
    whitespace_info = []
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    for col in string_cols:
        has_whitespace = df[col].astype(str).str.strip() != df[col].astype(str)
        if has_whitespace.any():
            count = has_whitespace.sum()
            whitespace_mask = has_whitespace
            examples_original = df[col][whitespace_mask].head(3).tolist()
            examples_cleaned = [str(val).strip() for val in examples_original]
            examples_orig_str = ' | '.join([f'"{val}"' for val in examples_original])
            examples_clean_str = ' | '.join([f'"{val}"' for val in examples_cleaned])
            whitespace_info.append({
                'column': col,
                'affected_rows': count,
                'percentage': round(count / len(df) * 100, 2),
                'examples_before': examples_orig_str,
                'examples_after': examples_clean_str
            })
    return pd.DataFrame(whitespace_info)

def trim_whitespace(df, columns=None):
    """Trim whitespace from specified columns (or all string columns if None)"""
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    if columns is not None:
        string_cols = [col for col in columns if col in string_cols]
    
    for col in string_cols:
        df[col] = df[col].str.strip()
    
    print(f"✓ Trimmed whitespace from {len(string_cols)} columns")
    return df

whitespace_issues = find_whitespace_in_values(df)
display(whitespace_issues)

if len(whitespace_issues) > 0:
    whitespace_columns = whitespace_issues['column'].tolist()
    df = trim_whitespace(df, whitespace_columns)

,column,affected_rows,percentage,examples_before,examples_after
0,Produit,3,6.0,"""Riz "" | ""Cafe "" | ""Jambon ""","""Riz"" | ""Cafe"" | ""Jambon"""
1,Notes,1,2.0,""" note à supprimer ""","""note à supprimer"""


✓ Trimmed whitespace from 2 columns


In [10]:
# Standardise strings - remove accents
df['Produit'] = (df['Produit']
    .str.lower()
    .str.replace('é', 'e', regex=False)
    .str.replace('è', 'e', regex=False)
    .str.replace('ê', 'e', regex=False)
    .str.replace('à', 'a', regex=False)
    .str.replace('â', 'a', regex=False)
    .str.replace('î', 'i', regex=False)
    .str.replace('ï', 'i', regex=False)
    .str.replace('ô', 'o', regex=False)
    .str.replace('ö', 'o', regex=False)
    .str.replace('û', 'u', regex=False)
    .str.replace('ü', 'u', regex=False)
    .str.replace('ç', 'c', regex=False)
)

## 5. Harmonisation des valeurs

In [11]:
def find_case_insensitive_duplicates(df):
    """
    Finds columns with case-insensitive duplicates (e.g., 'Apple', 'apple').
    Returns a DataFrame summarizing the issues for easy display.
    """
    results = []
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    for col in string_cols:
        series = df[col]
        clean_series = series.dropna().astype(str)
        if len(clean_series) == 0:
            continue
        case_map = {}
        for value in clean_series.unique():
            lower_val = value.lower()
            if lower_val in case_map:
                case_map[lower_val].append(value)
            else:
                case_map[lower_val] = [value]
        duplicate_groups = [group for group in case_map.values() if len(group) > 1]
        if duplicate_groups:
            value_counts = df[col].value_counts()
            total_affected_rows = 0
            example_groups = []
            for group in duplicate_groups:
                total_affected_rows += value_counts[group].sum()
                most_frequent_form = max(group, key=lambda x: value_counts.get(x, 0))
                group_str = ' | '.join([f'"{val}"' for val in sorted(group)])
                example_groups.append(f'{group_str} -> "{most_frequent_form}"')
            results.append({
                'column': col,
                'duplicate_groups': len(duplicate_groups),
                'affected_rows': total_affected_rows,
                'examples': ' || '.join(example_groups[:3])
            })
    return pd.DataFrame(results)

def standardise_case(df, columns: list):
    """
    Standardises the casing of values in the specified columns.
    """
    standardised_count = 0
    for col in columns:
        if col not in df.columns:
            continue
        series = df[col]
        clean_series = series.dropna().astype(str)
        if len(clean_series) == 0:
            continue
        case_map = {}
        for value in clean_series.unique():
            lower_val = value.lower()
            if lower_val in case_map:
                case_map[lower_val].append(value)
            else:
                case_map[lower_val] = [value]
        duplicate_groups = [group for group in case_map.values() if len(group) > 1]
        if not duplicate_groups:
            continue
        value_counts = df[col].value_counts()
        replacement_map = {}
        for group in duplicate_groups:
            most_frequent_form = max(group, key=lambda x: value_counts.get(x, 0))
            for variant in group:
                if variant != most_frequent_form:
                    replacement_map[variant] = most_frequent_form
        if replacement_map:
            df[col] = df[col].replace(replacement_map)
            standardised_count += 1
    
    print(f"✓ Standardised case in {standardised_count} columns")
    return df

case_insensitive = find_case_insensitive_duplicates(df)
if len(case_insensitive) > 0:
    case_insensitive_columns = case_insensitive['column'].tolist()
    df = standardise_case(df, case_insensitive_columns)

✓ Standardised case in 1 columns


In [12]:
df = standardise_case(df, ['Catégorie'])
df['Catégorie'].unique()

✓ Standardised case in 0 columns


array(['Boulangerie', 'Laitage', 'Fruits & Légumes', 'fruits et legumes',
       'Épicerie', 'epicerie', 'Œufs & Ovoproduits', 'oeufs', 'Boucherie',
       'Poissonnerie', 'fruits-legumes', 'Fruits/Légumes', 'Boissons',
       'boisson', nan, 'Crèmerie', 'Cremerie', 'Charcuterie', 'Divers'],
      dtype=object)

In [13]:
# Treat fuzzy duplicates
import re
from rapidfuzz import process, fuzz

def normalise_for_comparison(s: str) -> str:
    """Intelligently cleans a string for a base similarity comparison."""
    if not isinstance(s, str):
        return ""
    s_lower = s.lower()
    s_lower = re.sub(r'tbc\s*\(proposition\s*-?|local\s*|à\s*confirmer|pp\s*\d', '', s_lower)
    s_lower = re.sub(r'[\s-]+', '', s_lower)
    s_lower = s_lower.strip("()[]{}'\"- ")
    return s_lower

def find_fuzzy_duplicates(df, threshold: int = 85, min_length: int = 3):
    """
    Finds groups of similar strings (potential typos) in categorical columns.
    """
    issue_list = []
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    for col in string_cols:
        series = df[col]
        if series.nunique() < 2 or series.nunique() > 2000:
            continue
        categories = series.dropna().unique().tolist()
        filtered_cats = [
            cat for cat in set(categories)
            if isinstance(cat, str) and len(cat) >= min_length and not re.search(r'\d', cat)
        ]
        if len(filtered_cats) < 2:
            continue
        normalised_cats = [normalise_for_comparison(cat) for cat in filtered_cats]
        score_matrix = process.cdist(normalised_cats, normalised_cats, scorer=fuzz.ratio, score_cutoff=threshold)
        groups = []
        processed_indices = set()
        for i in range(len(filtered_cats)):
            if i in processed_indices:
                continue
            nonzero_result = score_matrix[i].nonzero()
            if isinstance(nonzero_result, tuple) and len(nonzero_result) > 0:
                similar_indices = nonzero_result[0] if len(nonzero_result) == 1 else nonzero_result[1]
            else:
                continue
            if len(similar_indices) > 1:
                current_group = {filtered_cats[j] for j in similar_indices}
                groups.append(sorted(list(current_group)))
                processed_indices.update(similar_indices)
        if groups:
            issue_list.append({'column': col, 'fuzzy_groups': groups})
    return issue_list

def standardise_fuzzy_values(df, column: str, mappings: dict):
    """
    Standardises values in a column based on a provided mapping.
    """
    if column in df.columns and mappings:
        df[column] = df[column].replace(mappings)
        print(f"✓ Applied {len(mappings)} fuzzy mappings to column '{column}'")
    return df

fuzzy_duplicates = find_fuzzy_duplicates(df)
display(fuzzy_duplicates)

for fuzzy_dict in fuzzy_duplicates:
    column_name = fuzzy_dict['column']
    fuzzy_groups = fuzzy_dict['fuzzy_groups']
    
    # Create mappings: all variants map to the first value in each group
    mappings = {}
    for group in fuzzy_groups:
        if len(group) > 1:
            canonical_value = group[0]  # First value is the standard
            for variant in group[1:]:    # All other values are variants
                mappings[variant] = canonical_value
    
    # Standardize the column using the mappings
    df = standardise_fuzzy_values(df, column_name, mappings)


[{'column': 'Produit',
  'fuzzy_groups': [['yaourt', 'yaourts'],
   ["huile d'olive", 'huile olive'],
   ['pomme', 'pommes'],
   ['choco lait', 'chocolat'],
   ['corn flakes', 'corn-flakes'],
   ['coca cola', 'coca-cola'],
   ['eau  minerale', 'eau minerale'],
   ['fromage', 'frommage'],
   ['oeuf', 'oeufs'],
   ['banane', 'bananes']]},
 {'column': 'Catégorie',
  'fuzzy_groups': [['Cremerie', 'Crèmerie'],
   ['epicerie', 'Épicerie'],
   ['Fruits & Légumes', 'Fruits/Légumes', 'fruits-legumes'],
   ['Boissons', 'boisson'],
   ['fruits et legumes', 'fruits-legumes']]}]

✓ Applied 10 fuzzy mappings to column 'Produit'
✓ Applied 5 fuzzy mappings to column 'Catégorie'


In [14]:
# Treat fuzzy duplicates
mapping_products = {
    'tomato': 'tomate',
}

mapping_category = {
    'fruits et legumes': 'Fruits & Légumes',
    'oeufs': 'Œufs & Ovoproduits',
    'epicerie': 'Epicerie'
}

# # Apply mapping
df['Produit'] = df['Produit'].replace(mapping_products)
df['Catégorie'] = df['Catégorie'].replace(mapping_category)

print(f"\nCatégies uniques après harmonisation: {df['Catégorie'].unique()}")
print(f"\nProduits uniques après harmonisation: {df['Produit'].unique()}")


Catégies uniques après harmonisation: ['Boulangerie' 'Laitage' 'Fruits & Légumes' 'Epicerie'
 'Œufs & Ovoproduits' 'Boucherie' 'Poissonnerie' 'Boissons' nan 'Cremerie'
 'Charcuterie' 'Divers']

Produits uniques après harmonisation: ['pain' 'lait' 'beurre' 'tomate' 'pates' 'riz' 'yaourt' 'oeuf' 'poulet'
 'poisson' 'banane' 'pomme' 'concombre' 'coca cola' 'eau  minerale' 'cafe'
 'the' 'choco lait' 'fromage' 'jambon' '—' "huile d'olive" 'corn flakes'
 'beurre demi-sel']


## 6. Unification du format des dates

In [15]:
print("=== Unification des dates ===")
print(f"\nExemples de dates avant conversion:")
print(df['Date'].head(20))
print(f"Type de la colonne Date: {df['Date'].dtype}")

# Function to parse dates with multiple formats
def parse_mixed_dates(date_str):
    """
    Parse dates with mixed formats:
    - YYYY-MM-DD, YYYY/MM/DD (year first)
    - DD/MM/YYYY, DD-MM-YYYY, DD/MM/YY (day first)
    """
    if pd.isna(date_str):
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    try:
        return pd.to_datetime(date_str)
    except:
        pass
    
    # Otherwise try day-first format (DD/MM/YYYY, DD-MM-YYYY, DD/MM/YY)
    return pd.to_datetime(date_str, dayfirst=True, errors='coerce')

# Apply the parsing function
print("\n--- Conversion des dates en cours ---")
df['Date'] = df['Date'].apply(parse_mixed_dates)

print(f"\nExemples de dates après conversion:")
print(df['Date'].head(20))
print(f"Type de la colonne Date: {df['Date'].dtype}")

# Check for unconverted dates (NaT)
invalid_dates = df['Date'].isna().sum()
print(f"\nNombre de dates invalides: {invalid_dates}")

if invalid_dates > 0:
    print("\nDates qui n'ont pas pu être converties:")
    print(df[df['Date'].isna()][['TransactionID', 'Produit', 'Date']])

=== Unification des dates ===

Exemples de dates avant conversion:
0     2025-09-01
1     01/09/2025
2     2025/09/01
3     2025-09-02
4     02-09-2025
5     2025-09-02
6       02/09/25
7     2025-09-03
8     2025-09-03
9     2025-09-03
10    03/09/2025
11    2025-09-04
12    04-09-2025
13    2025-09-04
14    2025-09-04
15    2025-09-05
16    05/09/2025
17    2025/09/05
18    2025-09-05
19    2025-09-06
Name: Date, dtype: object
Type de la colonne Date: object

--- Conversion des dates en cours ---

Exemples de dates après conversion:
0    2025-09-01
1    2025-01-09
2    2025-09-01
3    2025-09-02
4    2025-02-09
5    2025-09-02
6    2025-02-09
7    2025-09-03
8    2025-09-03
9    2025-09-03
10   2025-03-09
11   2025-09-04
12   2025-04-09
13   2025-09-04
14   2025-09-04
15   2025-09-05
16   2025-05-09
17   2025-09-05
18   2025-09-05
19   2025-09-06
Name: Date, dtype: datetime64[ns]
Type de la colonne Date: datetime64[ns]

Nombre de dates invalides: 0


## 7. Gestion des valeurs manquantes

In [16]:
print("=== Gestion des valeurs manquantes ===")
print(f"\nValeurs manquantes avant traitement:")
print(df[['Quantité', 'Prix']].isnull().sum())

# Statistics before imputation
print(f"\nStatistiques Quantité avant imputation:")
print(df['Quantité'].describe())
print(f"\nStatistiques Prix avant imputation:")
print(df['Prix'].describe())

# Impute missing values (median)
median_quantity = df['Quantité'].median()
median_price = df['Prix'].median()

print(f"\nValeur médiane pour Quantité: {median_quantity}")
print(f"Valeur médiane pour Prix: {median_price}")

# Fill empties
nb_missing_quantity = df['Quantité'].isnull().sum()
nb_missing_price = df['Prix'].isnull().sum()

df['Quantité'].fillna(median_quantity, inplace=True)
df['Prix'].fillna(median_price, inplace=True)

print(f"\nNombre de valeurs imputées:")
print(f"  - Quantité: {nb_missing_quantity}")
print(f"  - Prix: {nb_missing_price}")

print(f"\nValeurs manquantes après traitement:")
print(df[['Quantité', 'Prix']].isnull().sum())

=== Gestion des valeurs manquantes ===

Valeurs manquantes avant traitement:
Quantité    2
Prix        2
dtype: int64

Statistiques Quantité avant imputation:
count      48.000000
mean       23.520833
std       143.966159
min        -1.000000
25%         1.000000
50%         2.000000
75%         4.250000
max      1000.000000
Name: Quantité, dtype: float64

Statistiques Prix avant imputation:
count     48.000000
mean      22.756458
std      143.916366
min        0.000000
25%        0.897500
50%        1.575000
75%        2.825000
max      999.000000
Name: Prix, dtype: float64

Valeur médiane pour Quantité: 2.0
Valeur médiane pour Prix: 1.575

Nombre de valeurs imputées:
  - Quantité: 2
  - Prix: 2

Valeurs manquantes après traitement:
Quantité    0
Prix        0
dtype: int64


C:\Users\zahor\AppData\Local\Temp\ipykernel_26604\2178208705.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Quantité'].fillna(median_quantity, inplace=True)
C:\Users\zahor\AppData\Local\Temp\ipykernel_26604\2178208705.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [17]:
print("=== Gestion des valeurs manquantes ===")
print(f"\nValeurs manquantes avant traitement:")
print(df['Catégorie'].isnull().sum())

# Examine rows with missing categories
missing_cat_mask = df['Catégorie'].isnull()
print(f"\nLignes avec catégorie manquante:")
print(df[missing_cat_mask][['Produit', 'Catégorie']])

product_category_map = (
    df[df['Catégorie'].notna()]
    .groupby('Produit')['Catégorie']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
    .to_dict())

print(f"\nMapping Produit -> Catégorie créé avec {len(product_category_map)} produits")

# Fill missing categories based on product
for idx in df[missing_cat_mask].index:
    product = df.loc[idx, 'Produit']
    if pd.notna(product) and product in product_category_map:
        df.loc[idx, 'Catégorie'] = product_category_map[product]
        print(f"  Rempli: {product} -> {product_category_map[product]}")

# Remove rows that still have missing categories (no product or unknown product)
rows_before = len(df)
df = df[df['Catégorie'].notna()]
rows_after = len(df)
print(f"\nLignes supprimées (catégorie toujours manquante): {rows_before - rows_after}")

print(f"\nValeurs manquantes après traitement:")
print(df['Catégorie'].isnull().sum())

=== Gestion des valeurs manquantes ===

Valeurs manquantes avant traitement:
2

Lignes avec catégorie manquante:
   Produit Catégorie
35  tomate       NaN
49       —       NaN

Mapping Produit -> Catégorie créé avec 24 produits
  Rempli: tomate -> Fruits & Légumes
  Rempli: — -> Divers

Lignes supprimées (catégorie toujours manquante): 0

Valeurs manquantes après traitement:
0


## 8. Détection et traitement des valeurs aberrantes

In [18]:
print("=== Détection des valeurs aberrantes ===")

# Negative quantity
negative_quantity = (df['Quantité'] < 0).sum()
print(f"\nQuantités négatives: {negative_quantity}")
if negative_quantity > 0:
    print(df[df['Quantité'] < 0][['TransactionID', 'Produit', 'Quantité']])

# Quantity > 100
excessive_quantity = (df['Quantité'] > 100).sum()
print(f"\nQuantités excessives (>100): {excessive_quantity}")
if excessive_quantity > 0:
    print(df[df['Quantité'] > 100][['TransactionID', 'Produit', 'Quantité']])

# Outlier prices (> 100)
outlier_prices = (df['Prix'] > 100).sum()
print(f"\nPrix aberrants (>100): {outlier_prices}")
if outlier_prices > 0:
    print(df[df['Prix'] > 100][['TransactionID', 'Produit', 'Prix']])

# Empty products
empty_products = (df['Produit'] == '—').sum() + (df['Produit'] == '').sum()
print(f"\nProduits vides ou '—': {empty_products}")

=== Détection des valeurs aberrantes ===

Quantités négatives: 1
    TransactionID Produit  Quantité
31             32    lait      -1.0

Quantités excessives (>100): 1
    TransactionID Produit  Quantité
32             33    pain    1000.0

Prix aberrants (>100): 1
    TransactionID Produit   Prix
33             34   pates  999.0

Produits vides ou '—': 2


In [19]:
print("=== Traitement des valeurs aberrantes ===")
len_before_cleaning = len(df)

# Negative quantity: convert into NaN, then fill with median
df.loc[df['Quantité'] < 0, 'Quantité'] = pd.NA
nb_corrected_neg_quantity = df['Quantité'].isna().sum()

# Excessive quantity: convert into NaN, then fill with median
df.loc[df['Quantité'] > 100, 'Quantité'] = pd.NA
nb_corrected_exc_quantity = df['Quantité'].isna().sum() - nb_corrected_neg_quantity

# Impute outlier quantities
df['Quantité'].fillna(median_quantity, inplace=True)

# Outlier prices
df.loc[df['Prix'] > 100, 'Prix'] = pd.NA
nb_corrected_price_outlier = df['Prix'].isna().sum()

# Impute outlier prices
df['Prix'].fillna(median_price, inplace=True)

# Delete empties
df = df[df['Produit'] != '—']
df = df[df['Produit'] != '']
df = df[df['Produit'] != 'nan']

len_after_cleaning = len(df)
nb_deleted = len_before_cleaning - len_after_cleaning

print(f"\nNombre de lignes supprimées (produits invalides): {nb_deleted}")
print(f"Nombre total de lignes restantes: {len_after_cleaning}")

=== Traitement des valeurs aberrantes ===

Nombre de lignes supprimées (produits invalides): 2
Nombre total de lignes restantes: 48


C:\Users\zahor\AppData\Local\Temp\ipykernel_26604\2581040013.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Quantité'].fillna(median_quantity, inplace=True)
C:\Users\zahor\AppData\Local\Temp\ipykernel_26604\2581040013.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

## 9. Suppression de la colonne 'Notes'

In [20]:
print("=== Suppression de la colonne Notes ===")
print(f"\nColonnes avant suppression: {df.columns.tolist()}")

# Delete columns
if 'Notes' in df.columns:
    df = df.drop(columns=['Notes'])

print(f"Colonnes après suppression: {df.columns.tolist()}")

=== Suppression de la colonne Notes ===

Colonnes avant suppression: ['TransactionID', 'Produit', 'Quantité', 'Prix', 'Catégorie', 'Date', 'Notes']
Colonnes après suppression: ['TransactionID', 'Produit', 'Quantité', 'Prix', 'Catégorie', 'Date']


## 10. Vérification finale et export

In [21]:
print("=== Vérification finale ===")
print(f"\nDimensions du dataset final: {df.shape}")
print(f"\nAperçu du dataset nettoyé:")
print(df.head(10))

print(f"\n=== Informations finales ===")
df.info()

print(f"\n=== Valeurs manquantes finales ===")
print(df.isnull().sum())

print(f"\n=== Statistiques finales ===")
print(df.describe())

=== Vérification finale ===

Dimensions du dataset final: (48, 6)

Aperçu du dataset nettoyé:
   TransactionID Produit  Quantité  Prix         Catégorie       Date
0              1    pain       1.0  1.20       Boulangerie 2025-09-01
1              2    lait       2.0  0.95           Laitage 2025-01-09
2              3  beurre       1.0  2.80           Laitage 2025-09-01
3              4  tomate       3.0  1.99  Fruits & Légumes 2025-09-02
4              5  tomate       2.0  2.10  Fruits & Légumes 2025-02-09
5              6   pates       1.0  0.89          Epicerie 2025-09-02
6              7   pates       2.0  0.89          Epicerie 2025-02-09
7              8     riz       1.0  1.10          Epicerie 2025-09-03
8              9     riz       5.0  1.10          Epicerie 2025-09-03
9             10  yaourt       6.0  0.45           Laitage 2025-09-03

=== Informations finales ===
<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 0 to 50
Data columns (total 6 columns):
 #   Colu

In [22]:
# Export cleaned df
df.to_excel('donnees_achats_propre.xlsx', index=False)
print("\nFichier 'donnees_achats_propre.xlsx' exporté avec succès!")

# Export in csv
df.to_csv('donnees_achats_propre.csv', index=False, encoding='utf-8-sig')
print("Fichier 'donnees_achats_propre.csv' exporté avec succès!")


Fichier 'donnees_achats_propre.xlsx' exporté avec succès!
Fichier 'donnees_achats_propre.csv' exporté avec succès!
